In [1]:
%cd /nfs/dust/atlas/user/bauckhag/code/DESY-ATLAS-BSM/nn_studies/source

import sys
import os

sys.path.append("libs")

import math

import numpy as np
import matplotlib.pyplot as plt

import h5py


import ROOT
from ROOT import TCanvas, TPad, TFile, TPaveText, gStyle, TLorentzVector, TLegend, TEfficiency, TCollection, gPad
%jsroot on

from libs.TreeHolder import TreeHolder
from libs.NNimageMaker import GNNgraphMaker
from libs.NNdatasetConverter import GNNdatasetConverter

data_dir = "/nfs/dust/atlas/user/bauckhag/code/DESY-ATLAS-BSM/nn_studies/data/"

/nfs/dust/atlas/user/bauckhag/code/DESY-ATLAS-BSM/nn_studies/source
Welcome to JupyROOT 6.30/02


In [2]:
#load tree and print branches
file = TFile.Open(data_dir+"signal.root")
tree = file.Get("T")
branches = tree.GetListOfBranches()
for branch in branches:
    print(branch.GetName())

isMC
RunNumber
dsid
eventNumber
amiXsection
filterEff
kFactor
sumWeightPRW
lead_index
far20_index
etaLJ
phiLJ
ptLJ
types
isoID
LJ_index
LJ_MatchedTruthDPindex
weight
puWeight
mcWeight
LJjet_index
LJjet_eta
LJjet_phi
LJjet_pt
LJjet_width
LJjet_EMfrac
LJjet_timing
LJjet_jvt
LJjet_gapRatio
LJjet_IsBIB
LJjet_m
truthPdgId
truthEta
truthPhi
truthPt
truthE
truthCharge
truthBarcode
truthDecayVtx_x
truthDecayVtx_y
truthDecayVtx_z
truthDecayType
childPdgId
childEta
childPhi
childPt
childBarcode
childMomBarcode
GNNjet_eta
GNNjet_phi
GNNjet_pt
GNNjet_width
GNNjet_EMfrac
GNNjet_timing
GNNjet_jvt
GNNjet_gapRatio
GNNjet_m
GNNjet_clusESampl
GNNjet_clusSamplIndex
GNNjet_clusEta
GNNjet_clusPhi


In [3]:
#some studies on example graphs
file = TFile.Open(data_dir+"signal.root")

tree = file.Get("T")
tree_reader = TreeHolder(tree)

graph_maker = GNNgraphMaker()

nentries = tree.GetEntries()
entry = 1    

tree.GetEntry(entry)
tree_reader.update()

if entry%1000 == 0:
    print("entry", str(entry) + '/' + str(nentries), "("+str(round(entry/nentries*100, 2))+"%)")

print("n_jets=",tree_reader.jet_width.size())
for pj in range(tree_reader.jet_width.size()):
    jet_graph = graph_maker.MakeGraph(tree_reader, pj)
    if not (type(jet_graph) is list):
        print("ERROR: pj =", pj, "entry =", entry, "code =", jet_graph)
    else:
        layers = ["bar"]
        print(jet_graph)
        for i in range(len(jet_graph)):
            print(jet_graph[i])
            #for j in range(len(jet_graph)):
            #    print(jet_graph[j][i])
            
# jet_graph = [nodes_posbar, nodes_posbar2, nodes_etabar, nodes_phibar, nodes_layerbar, nodes_energybar, nodes_energybarabs]
# nodes_posbar = [nodes_posbar_layer0, nodes_posbar_layer1, nodes_posbar_layer2, nodes_posbar_layer3]
# nodes_posbar_layer0 = {0: vbar_0, ..., n_nodes-1: vbar_n_nodes-1}

n_jets= 1
[[{0: [1e-08, 1e-08], 1: [1e-08, 1e-08], 2: [-0.3752385377883911, -0.09061169624328613], 3: [1e-08, 1e-08], 4: [1e-08, 1e-08], 5: [1e-08, 1e-08], 6: [0.30326104164123535, 0.22626638412475586], 7: [1e-08, 1e-08], 8: [1e-08, 1e-08], 9: [1e-08, 1e-08], 10: [0.21283304691314697, 0.3083205223083496], 11: [1e-08, 1e-08], 12: [1e-08, 1e-08], 13: [1e-08, 1e-08], 14: [0.2987861633300781, 0.052980899810791016]}, {0: [1e-08, 1e-08], 1: [1e-08, 1e-08], 2: [-0.3752385377883911, -0.09061169624328613], 3: [1e-08, 1e-08], 4: [1e-08, 1e-08], 5: [1e-08, 1e-08], 6: [0.30326104164123535, 0.22626638412475586], 7: [1e-08, 1e-08], 8: [1e-08, 1e-08], 9: [1e-08, 1e-08], 10: [0.21283304691314697, 0.3083205223083496], 11: [1e-08, 1e-08], 12: [1e-08, 1e-08], 13: [1e-08, 1e-08], 14: [0.2987861633300781, 0.052980899810791016]}, {0: [1e-08, 1e-08], 1: [1e-08, 1e-08], 2: [-0.3752385377883911, -0.09061169624328613], 3: [1e-08, 1e-08], 4: [1e-08, 1e-08], 5: [1e-08, 1e-08], 6: [0.30326104164123535, 0.226266384

In [5]:
#dataset converter
import deepdish as dd

output_dir = data_dir + "graphs/raw/Signal_v6/"

n_layers = 4
labels = ["barnodes", "barpos3", "barpos4", "bareta", "barphi", "barlayer", "barenergy", "barenergyabs"]

file = TFile.Open(data_dir+"signal.root")

tree = file.Get("T")
tree_reader = TreeHolder(tree)

graph_maker = GNNgraphMaker()

jet_graphs = []

nentries = tree.GetEntries()
for entry in range(nentries):
    tree.GetEntry(entry)
    tree_reader.update()

    if entry%(int(nentries/100)) == 0:
        print("entry", str(entry) + '/' + str(nentries), "("+str(round(entry/nentries*100, 2))+"%)", end='\r')

    #print("n_jets=",tree_reader.jet_width.size())
    for pj in range(tree_reader.jet_width.size()):
        jet_graph = graph_maker.MakeGraph(tree_reader, pj)
        if not (type(jet_graph) is list):
            print("ERROR: pj =", pj, "entry =", entry, "code =", jet_graph)
        else:
            layers = ["bar"]
            jet_graphs.append(jet_graph)


for i in range(len(labels)):
    for j in range(n_layers):
        filename = output_dir + 'a' + str(j) + "_tupleGraph_" + labels[i] + '.h5'
        dd.io.save(filename, jet_graphs[:][i][j])

In [2]:
#dataset converter
filename = "background2.root"
output_dir = data_dir + "graphs/raw/Background_v6/"
input_dir = data_dir

converter = GNNdatasetConverter()

converter.convert_dataset(filename, input_dir, output_dir)